In [1]:
!pip install -e .

Obtaining file:///Users/ydchen/MBDS_FYP/FYP_v3
  Preparing metadata (setup.py) ... done
  Attempting uninstall: myproject
    Found existing installation: myproject 1.0
    Uninstalling myproject-1.0:
      Successfully uninstalled myproject-1.0
  Running setup.py develop for myproject


In [51]:
from Crypto.Hash import keccak
def obtain_hash_event(event: str) -> str:
    k = keccak.new(digest_bits=256)
    k.update(bytes(event, encoding='utf-8'))
    return '0x'+k.hexdigest()
# obtain_hash_event('PairCreated(address,address,address,uint256)')

In [14]:
# Transfer_hash = obtain_hash_event('Transfer(address,address,uint256)')
# Burn_hash = obtain_hash_event('Burn(address,uint256,uint256,address)')
# Mint_hash = obtain_hash_event("Mint(address,uint256,uint256)")
# Sync_hash = obtain_hash_event('Sync(uint112,uint112)')

'0x0d3648bd0f6ba80134a33ba9275ac585d9d315f0ad8355cddefde31afa28d0e9'

In [10]:
import time
import pandas as pd
import requests
import shared
shared.init()
import json
from web3 import Web3, HTTPProvider
from hexbytes import HexBytes
from web3.datastructures import AttributeDict
from Utils.eth_utils import  obtain_hash_event

def connect_to_web3():
    """
    Connect to Web3 server.
    Args:
    Returns:
      res: Boolean indicating that connection was succeed or not.
      web3: Web3 Object
    """
    web3 = Web3(HTTPProvider('https://polygon-mainnet.infura.io/v3/d6243bb783b44485ad6636b6c3411377'))
    res = web3.isConnected()
    return res, web3

from_block = 22757547
response = [0] * 1000
block_number = []
_, web3 = connect_to_web3()
contract = web3.eth.contract(shared.UNISWAP_V2_FACTORY, abi="""[{"inputs":[{"internalType":"address","name":"_feeToSetter","type":"address"}],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"token0","type":"address"},{"indexed":true,"internalType":"address","name":"token1","type":"address"},{"indexed":false,"internalType":"address","name":"pair","type":"address"},{"indexed":false,"internalType":"uint256","name":"","type":"uint256"}],"name":"PairCreated","type":"event"},{"constant":true,"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"allPairs","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"allPairsLength","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"tokenA","type":"address"},{"internalType":"address","name":"tokenB","type":"address"}],"name":"createPair","outputs":[{"internalType":"address","name":"pair","type":"address"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"feeTo","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"feeToSetter","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"address","name":"","type":"address"}],"name":"getPair","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"_feeTo","type":"address"}],"name":"setFeeTo","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"_feeToSetter","type":"address"}],"name":"setFeeToSetter","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}]""")

topic = obtain_hash_event('PairCreated(address,address,address,uint256)')
# topic = obtain_hash_event('PoolCreated(address,address,uint24,int24,address')


tokens0, tokens1, pairs, block_numbers, timestamps = [], [], [], [], []
cont = 1000
while len(response) >= 1000:
    try:
        print("Pools trobades", cont)
        cont += 1000
        apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"
        endpoint = "https://api.polygonscan.com/api?module=logs" \
                   "&action=getLogs" \
                   f"&fromBlock={from_block}" \
                   f"&toBlock={shared.BLOCKSTUDY}" \
                   f"&address={shared.SUSHISWAP_FACTORY}" \
                   f"&topic0={topic}" \
                   f"&apikey={apikey}"

        response = json.loads(requests.get(endpoint).text)['result']

        # print(response)
        receipts = []
        for event in response:
            receipts.append(AttributeDict({
                'blockNumber': int(event['blockNumber'], 16),
                'contractAddress': event['address'],
                'topics': [HexBytes(topic) for topic in event['topics']],
                'transactionHash': HexBytes(event['transactionHash']),
                'data': event['data'],
                'logIndex': 0,
                'transactionIndex': event['transactionIndex'],
                'address': event['address'],
                'blockHash': HexBytes(event['transactionHash'])
            }))
        log_dict = {'logs': receipts}
        # print(log_dict)
        decoded_logs = contract.events.PairCreated().processReceipt(log_dict)
        # print(decoded_logs)
        for k, log in enumerate(decoded_logs):
            block_numbers.append(log['blockNumber'])
            tokens0.append(log['args']["token0"])
            tokens1.append(log['args']['token1'])
            pairs.append(log['args']['pair'])
            #fees.append(log['args']['fee'])
            timestamps.append(int(response[k]['timeStamp'], 16))
        from_block = int(response[-1]['blockNumber'], 16) + 1

    except Exception as err:
        print("error", err)
        time.sleep(2)

pd.DataFrame({'pair': pairs, 'token0': tokens0, 'token1': tokens1, 'block_number': block_numbers, 'timestamp': timestamps}).to_csv(
    "./data/uniswap_v2/polygon_pools.csv", index=False)

Pools trobades 1000
Pools trobades 2000


In [17]:
import os
import time
import pandas as pd
import requests
import shared
from Utils.eth_utils import obtain_hash_event, connect_to_web3
shared.init()
import json
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

def get_response(pool, from_block):
    endpoint = "https://api.polygonscan.com/api?module=logs" \
               "&action=getLogs" \
               f"&fromBlock={from_block}" \
               f"&toBlock={shared.BLOCKSTUDY}" \
               f"&address={pool}" \
               f"&topic0={sync_hash}" \
               f"&apikey={apikey}"

    return json.loads(requests.get(endpoint).text)['result']

def clean_response(response):
    receipts = []
    for event in response:
        receipts.append(AttributeDict({
            'blockNumber': int(event['blockNumber'], 16),
            'contractAddress': event['address'],
            'topics': [HexBytes(topic) for topic in event['topics']],
            'transactionHash': HexBytes(event['transactionHash']),
            'data': event['data'],
            'logIndex': 0,
            'transactionIndex': event['transactionIndex'],
            'address': event['address'],
            'blockHash': HexBytes(event['transactionHash'])
        }))

    log_dict = {'logs': receipts}
    return contract.events.Sync().processReceipt(log_dict)

sync_hash = obtain_hash_event('Sync(uint112,uint112)')
_, web3 = connect_to_web3()
apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"
tokens_and_pools = pd.read_csv("./data/uniswap_v2/polygon_pools.csv", index_col='pair')
pools = tokens_and_pools.index.tolist()

contract = web3.eth.contract("0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D", abi=shared.ABI_POOL)
done_pools = [pool.split(".csv")[0] for pool in os.listdir("./data/uniswap_v2/sync")]

for i, pool in enumerate(pools):
    if pool in done_pools:
        continue
    response = [0] * 1000

    print(pool, i, len(pools))
    timestamps, block_numbers, reserves0, reserves1 = [], [], [], []
    from_block, first_timestamp, current_timestamp = 0, 0, 0
    cont = 0

    while len(response) >= 1000 and current_timestamp - first_timestamp < shared.WEEK:
        response = get_response(pool, from_block)
        while response == None or "Max rate limit" in response:
            time.sleep(20)
            response = get_response(pool, from_block)
            if response != None:
                break
        if len(response) == 0:
            continue
        try:
            decoded_logs = clean_response(response)
        except:
            print(pool, i, len(pools))
            print(response)
        for k, log in enumerate(decoded_logs):
            block_numbers.append(log['blockNumber'])
            reserves0.append(log['args']["reserve0"])
            reserves1.append(log['args']["reserve1"])
            timestamps.append(int(response[k]["timeStamp"], 16))
        from_block = int(response[-1]['blockNumber'], 16) + 1
        current_timestamp = int(response[-1]['timeStamp'], 16)

    pd.DataFrame({'block_number': block_numbers, 'reseve0': reserves0 , 'reserve1': reserves1, 'timestamps': timestamps}).to_csv(
        f"./data/uniswap_v2/sync/{pool}.csv", index=False)

0x1Fec149e363b9c3282C2C3509f171cE2Ac77dd27 500 1459
0x12Bed9dE2d68538Ae20BeDBbe7812d450c6cE51c 501 1459
0xDb101f7b300Ca1d1910A38A3a0EA3c75A39c509a 502 1459
0x4e8777B91a3DA3b1a57c697866D7f07e9C1b67a1 503 1459
0xf843C9af2770988B0Eb7Bf8c3844df6BFaeff4c7 504 1459
0xF509cDC34Bc5521bE13A906942fc426A74ddc2bf 505 1459
0x99FF9C60f8Ed3e0906472591989063C45072E68E 506 1459
0xdf06a7932a233404c8fA009d61523dDb4510aE0c 507 1459
0xF6bf583aD4a1D8777D180416a454788F517A18E9 508 1459
0x960c269c37f8075DE0b7175b6d4517CA4092DA43 509 1459
0x035E1Ff00D09fdd5CF9D652418feA0E57376aDa4 510 1459
0x72439387d0A6C06025F1B90C3D346de5b256AF4D 511 1459
0x254b8Aca5B65CEF657ADB8A12e98AAbbA9F6D3B6 512 1459
0x8fB4a2A93b8880e56F77F8a33F871e26e911247F 513 1459
0x9c6f75CB8cF745a19e0C754a6c440961561c63b2 514 1459
0xEE72030E79fa506ebDf25a58015949f1BE3b476C 515 1459
0x3195D70B99feF8c5e8Bee3e00e6D1Bd5D7c836d2 516 1459
0x8d16A4E80683Ef1212a5b6b17E2185eDfcEF7FA4 517 1459
0x5Dd1a6B0C4dB57F7e53DCC4549e46Fbf2bC52290 518 1459
0x2EEBe239DE

In [20]:
#!pip install web3_multicall

In [19]:
import os
import time
import pandas as pd
import requests
import shared
from Utils.eth_utils import obtain_hash_event, connect_to_web3
shared.init()
import json
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

def get_response(pool, from_block):
    endpoint = "https://api.polygonscan.com/api?module=logs" \
               "&action=getLogs" \
               f"&fromBlock={from_block}" \
               f"&toBlock={shared.BLOCKSTUDY}" \
               f"&address={pool}" \
               f"&topic0={sync_hash}" \
               f"&apikey={apikey}"

    return json.loads(requests.get(endpoint).text)['result']

def clean_response(response):
    receipts = []
    for event in response:
        receipts.append(AttributeDict({
            'blockNumber': int(event['blockNumber'], 16),
            'contractAddress': event['address'],
            'topics': [HexBytes(topic) for topic in event['topics']],
            'transactionHash': HexBytes(event['transactionHash']),
            'data': event['data'],
            'logIndex': 0,
            'transactionIndex': event['transactionIndex'],
            'address': event['address'],
            'blockHash': HexBytes(event['transactionHash'])
        }))

    log_dict = {'logs': receipts}
    return contract.events.Swap().processReceipt(log_dict)

sync_hash = obtain_hash_event('Swap(address,unit256,unit256,unit256,unit256,address)')
_, web3 = connect_to_web3()
apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"
tokens_and_pools = pd.read_csv("./data/uniswap_v2/polygon_pools.csv", index_col='pair')
pools = tokens_and_pools.index.tolist()

contract = web3.eth.contract("0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D", abi=shared.ABI_POOL)
done_pools = [pool.split(".csv")[0] for pool in os.listdir("./data/uniswap_v2/swap")]

for i, pool in enumerate(pools):
    if pool in done_pools:
        continue
    response = [0] * 1000

    print(pool, i, len(pools))
    timestamps, block_numbers, reserves0, reserves1 = [], [], [], []
    from_block, first_timestamp, current_timestamp = 0, 0, 0
    cont = 0

    while len(response) >= 1000 and current_timestamp - first_timestamp < shared.WEEK:
        response = get_response(pool, from_block)
        while response == None or "Max rate limit" in response:
            time.sleep(20)
            response = get_response(pool, from_block)
            if response != None:
                break
        if len(response) == 0:
            continue
        try:
            decoded_logs = clean_response(response)
        except:
            print(pool, i, len(pools))
            print(response)
        for k, log in enumerate(decoded_logs):
            block_numbers.append(log['blockNumber'])
            reserves0.append(log['args']["reserve0"])
            reserves1.append(log['args']["reserve1"])
            timestamps.append(int(response[k]["timeStamp"], 16))
        from_block = int(response[-1]['blockNumber'], 16) + 1
        current_timestamp = int(response[-1]['timeStamp'], 16)

    pd.DataFrame({'block_number': block_numbers, 'reseve0': reserves0 , 'reserve1': reserves1, 'timestamps': timestamps}).to_csv(
        f"./data/uniswap_v2/swap/{pool}.csv", index=False)

0x13bd20ba647E2ed12622851F652a5267869A8c62 0 1459
0x531eB8f8E8e626cCFaB2BAA38d14A45096a50180 1 1459
0x103C0675054D31FD081f53a15FE108c5B2A2f456 2 1459
0x2d8Db3B54bc4c08B295816a30ad74b01109e4e9C 3 1459
0x61637f75DCecEc29e11D73Bfa3ac698Dd5Ef8a0f 4 1459
0xfA69bF4fA125ef9Cc74945EAc914150e5b3cF3E0 5 1459
0xb874c2403391fc1571f778fb6fa9A909eb0E79b7 6 1459
0x4f414E6489fD19bBb50C1Bc6C403E19D0dC71Fd2 7 1459
0x1A3713dE5Fe72EDe1f5aA08d563Da6c5D4Bc2b9e 8 1459
0x53aDd4C98B2787f690042771Ca8E512a5793E9C9 9 1459
0x46c692346CEDCF46C7c9002680E2645c8577EB3a 10 1459
0x99219f505BfffBE1A58fAF113eBc861b175E41f7 11 1459
0x47F4947a1b44F773EDb4520E170621edf40b2f05 12 1459
0xCD3a618948256F77f794FAe7D30Dea18D5ca51A0 13 1459
0x0b43d8e38d02Ce15D54099A593852824D882cFE2 14 1459
0xE31461B5adCF2174ae53C64dB57C3F59CfED987b 15 1459
0x36B860b863C23a1564aD19F28FD4260443E007a2 16 1459
0x8D04E60fD7cEd96A24e726968F93397C5a467890 17 1459
0x53f858f28Fd9120bB6Ea1b7F5961333EBE734D22 18 1459
0xDeDef8a9ebd5Fb001A596348EB825f0b9EdaF12

In [21]:
import os
import time
import pandas as pd
import requests
import shared
from Utils.eth_utils import obtain_hash_event, connect_to_web3
shared.init()
import json
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

def get_response(pool, from_block):
    endpoint = "https://api.polygonscan.com/api?module=logs" \
               "&action=getLogs" \
               f"&fromBlock={from_block}" \
               f"&toBlock={shared.BLOCKSTUDY}" \
               f"&address={pool}" \
               f"&topic0={sync_hash}" \
               f"&apikey={apikey}"

    return json.loads(requests.get(endpoint).text)['result']

def clean_response(response):
    receipts = []
    for event in response:
        receipts.append(AttributeDict({
            'blockNumber': int(event['blockNumber'], 16),
            'contractAddress': event['address'],
            'topics': [HexBytes(topic) for topic in event['topics']],
            'transactionHash': HexBytes(event['transactionHash']),
            'data': event['data'],
            'logIndex': 0,
            'transactionIndex': event['transactionIndex'],
            'address': event['address'],
            'blockHash': HexBytes(event['transactionHash'])
        }))

    log_dict = {'logs': receipts}
    return contract.events.Mint().processReceipt(log_dict)

sync_hash = obtain_hash_event("Mint(address,uint256,uint256)")
_, web3 = connect_to_web3()
apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"
tokens_and_pools = pd.read_csv("./data/uniswap_v2/polygon_pools.csv", index_col='pair')
pools = tokens_and_pools.index.tolist()

contract = web3.eth.contract("0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D", abi=shared.ABI_POOL)
done_pools = [pool.split(".csv")[0] for pool in os.listdir("./data/uniswap_v2/mint")]

for i, pool in enumerate(pools):
    if pool in done_pools:
        continue
    response = [0] * 1000

    print(pool, i, len(pools))
    timestamps, block_numbers, sender ,amount0, amount1 = [], [], [], [], []
    from_block, first_timestamp, current_timestamp = 0, 0, 0
    cont = 0

    while len(response) >= 1000 and current_timestamp - first_timestamp < shared.WEEK:
        response = get_response(pool, from_block)
        while response == None or "Max rate limit" in response:
            time.sleep(20)
            response = get_response(pool, from_block)
            if response != None:
                break
        if len(response) == 0:
            continue
        try:
            decoded_logs = clean_response(response)
        except:
            print(pool, i, len(pools))
            print(response)
        for k, log in enumerate(decoded_logs):
            block_numbers.append(log['blockNumber'])
            sender.append(log['args']["sender"])
            amount0.append(log['args']["amount0"])
            amount1.append(log['args']["amount1"])
            timestamps.append(int(response[k]["timeStamp"], 16))
        
        from_block = int(response[-1]['blockNumber'], 16) + 1
        current_timestamp = int(response[-1]['timeStamp'], 16)

    pd.DataFrame({'block_number': block_numbers, 'sender': sender, 
                  'amount0':amount0,'amount1':amount1,'timestamps': timestamps}).to_csv(f"./data/uniswap_v2/mint/{pool}.csv", index=False)

0x13bd20ba647E2ed12622851F652a5267869A8c62 0 1459
0x531eB8f8E8e626cCFaB2BAA38d14A45096a50180 1 1459
0x103C0675054D31FD081f53a15FE108c5B2A2f456 2 1459
0x2d8Db3B54bc4c08B295816a30ad74b01109e4e9C 3 1459
0x61637f75DCecEc29e11D73Bfa3ac698Dd5Ef8a0f 4 1459
0xfA69bF4fA125ef9Cc74945EAc914150e5b3cF3E0 5 1459
0xb874c2403391fc1571f778fb6fa9A909eb0E79b7 6 1459
0x4f414E6489fD19bBb50C1Bc6C403E19D0dC71Fd2 7 1459
0x1A3713dE5Fe72EDe1f5aA08d563Da6c5D4Bc2b9e 8 1459
0x53aDd4C98B2787f690042771Ca8E512a5793E9C9 9 1459
0x46c692346CEDCF46C7c9002680E2645c8577EB3a 10 1459
0x99219f505BfffBE1A58fAF113eBc861b175E41f7 11 1459
0x47F4947a1b44F773EDb4520E170621edf40b2f05 12 1459
0xCD3a618948256F77f794FAe7D30Dea18D5ca51A0 13 1459
0x0b43d8e38d02Ce15D54099A593852824D882cFE2 14 1459
0xE31461B5adCF2174ae53C64dB57C3F59CfED987b 15 1459
0x36B860b863C23a1564aD19F28FD4260443E007a2 16 1459
0x8D04E60fD7cEd96A24e726968F93397C5a467890 17 1459
0x53f858f28Fd9120bB6Ea1b7F5961333EBE734D22 18 1459
0xDeDef8a9ebd5Fb001A596348EB825f0b9EdaF12

In [41]:
import os
import time
import pandas as pd
import requests
import shared
from Utils.eth_utils import obtain_hash_event, connect_to_web3
shared.init()
import json
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

def get_response(pool, from_block):
    endpoint = "https://api.polygonscan.com/api?module=logs" \
               "&action=getLogs" \
               f"&fromBlock={from_block}" \
               f"&toBlock={shared.BLOCKSTUDY}" \
               f"&address={pool}" \
               f"&topic0={sync_hash}" \
               f"&apikey={apikey}"

    return json.loads(requests.get(endpoint).text)['result']

def clean_response(response):
    receipts = []
    for event in response:
        receipts.append(AttributeDict({
            'blockNumber': int(event['blockNumber'], 16),
            'contractAddress': event['address'],
            'topics': [HexBytes(topic) for topic in event['topics']],
            'transactionHash': HexBytes(event['transactionHash']),
            'data': event['data'],
            'logIndex': 0,
            'transactionIndex': event['transactionIndex'],
            'address': event['address'],
            'blockHash': HexBytes(event['transactionHash'])
        }))

    log_dict = {'logs': receipts}
    return contract.events.Burn().processReceipt(log_dict)

sync_hash = obtain_hash_event('Burn(address,uint256,uint256,address)')
_, web3 = connect_to_web3()
apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"
tokens_and_pools = pd.read_csv("./data/uniswap_v2/polygon_pools.csv", index_col='pair')
pools = tokens_and_pools.index.tolist()

contract = web3.eth.contract("0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D", abi=shared.ABI_POOL)
done_pools = [pool.split(".csv")[0] for pool in os.listdir("./data/uniswap_v2/burn")]

for i, pool in enumerate(pools):
    if pool in done_pools:
        continue
    response = [0] * 1000

    print(pool, i, len(pools))
    timestamps, block_numbers, sender, to, amount0, amount1 = [], [], [], [], [], []
    from_block, first_timestamp, current_timestamp = 0, 0, 0
    cont = 0

    while len(response) >= 1000 and current_timestamp - first_timestamp < shared.WEEK:
        response = get_response(pool, from_block)
        while response == None or "Max rate limit" in response:
            time.sleep(20)
            response = get_response(pool, from_block)
            if response != None:
                break
        if len(response) == 0:
            continue
        try:
            decoded_logs = clean_response(response)
        except:
            print(pool, i, len(pools))
        # print(decoded_logs[0].keys())
        for k, log in enumerate(decoded_logs):
            # print(log['args'])
            block_numbers.append(log['blockNumber'])
            sender.append(log['args']["sender"])
            to.append(log['args']["to"])
            amount0.append(log['args']["amount0"])
            amount1.append(log['args']["amount1"])
            timestamps.append(int(response[k]["timeStamp"], 16))
        from_block = int(response[-1]['blockNumber'], 16) + 1
        current_timestamp = int(response[-1]['timeStamp'], 16)

    pd.DataFrame({'block_number': block_numbers, 'sender': sender , 'to': to, 'amount0':amount0, 'amount1':amount1}).to_csv(
        f"./data/uniswap_v2/burn/{pool}.csv", index=False)

0xb8fcA2597D56D02731D8BBd4eaDeBE64e0AC669e 27 1459
0x360a9a79827ea6273378383B717cdC3Ef653856C 28 1459
0x7aA9B3dd6785004C5FFd12867b1D549481387eF1 29 1459
0x33b95F2AeCa454229D6429bb60000A1B2Ac80Be2 30 1459
0x798bd8F580ae6fC704D94decb38Dd2FA61c33Bcc 31 1459
0xd22DD08FB7CdC9154F4354aF79754600f87b2858 32 1459
0x0759BdF7af649f5C9D66CC9C25c28f2cC0B374f4 33 1459
0x74f0Cf30aC8EDC5Ea5b20994bdA0f61c95969bB6 34 1459
0x6B9dB459A71DA984cA96f42f10dE0558Ef479281 35 1459
0x02E3C1C40F87FAf92793e3c090Dc68470c94c2A0 36 1459
0xC7371CD4D33778878a1fE2254F293893E641230d 37 1459
0xB259dD2307E7626d925efb6D523b986b670e5631 38 1459
0x848aFbed132BDAcdf0466A570E70c37777d8bAaF 39 1459
0xBf0f72a6315a351b20251f32C49E3Dec868a969B 40 1459
0xBDC85c0360B4f38Fe8e0b558b0031e5CA5003cB4 41 1459
0x967f6e52442709Fe3806Ee7fEA4020ACB9C50026 42 1459
0xcc2Ae5a788D2F20f8eced42F0A575Fa808285558 43 1459
0x3CC7c7DBBEA21EbbfB9739437936185aadbb1e2A 44 1459
0xeFcFeF98705aA7B310b2C85938f946c17bfbE7df 45 1459
0xa3dC7aA344F366B746d7B63c5168a

In [50]:
import os
import time
import pandas as pd
import requests
import shared
from Utils.eth_utils import obtain_hash_event, connect_to_web3
shared.init()
import json
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

def get_response(pool, from_block):
    endpoint = "https://api.polygonscan.com/api?module=logs" \
               "&action=getLogs" \
               f"&fromBlock={from_block}" \
               f"&toBlock={shared.BLOCKSTUDY}" \
               f"&address={pool}" \
               f"&topic0={sync_hash}" \
               f"&apikey={apikey}"
    try:
        result = json.loads(requests.get(endpoint).text)['result']
    except:
        print(requests.get(endpoint).text)
        result = None
    return result

def clean_response(response):
    receipts = []
    for event in response:
        receipts.append(AttributeDict({
            'blockNumber': int(event['blockNumber'], 16),
            'contractAddress': event['address'],
            'topics': [HexBytes(topic) for topic in event['topics']],
            'transactionHash': HexBytes(event['transactionHash']),
            'data': event['data'],
            'logIndex': 0,
            'transactionIndex': event['transactionIndex'],
            'address': event['address'],
            'blockHash': HexBytes(event['transactionHash'])
        }))

    log_dict = {'logs': receipts}
    return contract.events.Transfer().processReceipt(log_dict)

sync_hash = obtain_hash_event('Transfer(address,address,uint256)')
_, web3 = connect_to_web3()
apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"
tokens_and_pools = pd.read_csv("./data/uniswap_v2/polygon_pools.csv", index_col='pair')
pools = tokens_and_pools.index.tolist()

contract = web3.eth.contract("0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D", abi=shared.ABI_POOL)
done_pools = [pool.split(".csv")[0] for pool in os.listdir("./data/uniswap_v2/transfer")]

for i, pool in enumerate(pools):
    if pool in done_pools:
        continue
    response = [0] * 1000

    print(pool, i, len(pools))
    timestamps, block_numbers, fromx, to, value = [], [], [], [], []
    from_block, first_timestamp, current_timestamp = 0, 0, 0
    cont = 0

    while len(response) >= 1000 and current_timestamp - first_timestamp < shared.WEEK:
        response = get_response(pool, from_block)
        while response == None or "Max rate limit" in response:
            time.sleep(20)
            response = get_response(pool, from_block)
            if response != None:
                break
        if len(response) == 0:
            continue
        try:
            decoded_logs = clean_response(response)
        except:
            print(pool, i, len(pools))
            # print(response)
        # print(decoded_logs)
        for k, log in enumerate(decoded_logs):
            block_numbers.append(log['blockNumber'])
            fromx.append(log['args']["from"])
            to.append(log['args']["to"])
            value.append(log['args']["value"])
            timestamps.append(int(response[k]["timeStamp"], 16))
        from_block = int(response[-1]['blockNumber'], 16) + 1
        current_timestamp = int(response[-1]['timeStamp'], 16)

    pd.DataFrame({'block_number': block_numbers, 'from': fromx , 'to': to, "value":value,  'timestamps': timestamps}).to_csv(
        f"./data/uniswap_v2/transfer/{pool}.csv", index=False)

0x70a65Af6C249D4F72702C62e84D395aA890D9741 1056 1459
0xdaC3fc5Ca997E1b0D500a5E68ecaa9b47c6c5BDF 1057 1459
0x74Ffa1C5CE426bb018D875Fa7eba9eE5525Fd9D9 1058 1459
0x87A3d8C6eB96021cD92dCb8433b7645b5f8F8537 1059 1459
0xd852d74e6Ad074AFBe9E8bc51307B0fB594D4E0B 1060 1459
0x31f5d660681fACe860aE2938b076CD4E2f25938A 1061 1459
0x7120C7F32D6ED820223c4bB915AE4491BE01D940 1062 1459
0x7e04DB926F795C11921071edd91deE70fb68f610 1063 1459
0x8F0bf3118d9A3e7DeF7b7ea1eaB699f958AFA36F 1064 1459
0x16d4618717B0A3C7D7Cc261110e61A02E7FB0b73 1065 1459
0xfd361D6AcF8C49FCb375fe20ff944125d986c029 1066 1459
0x490e3F5a38765B9e8cC971574afd8539037db971 1067 1459
0x6CD445E398b346342164A550070AE7a77029385a 1068 1459
0xDE5A01427bDd05fd84d54bBbdbC75dA4cbB2Df51 1069 1459
0x221Fa4080C7AA3c913e8E77713b70E50DfAE983D 1070 1459
0xEF1EdFbc07Cb47821F31dac6f147131e6CF8A23b 1071 1459
0xFCA78B2c5966ED4655860B780b8866E1c81ac8e7 1072 1459
0x91CB51fbDF1384F678463a23CB54A0F0eA8B9c53 1073 1459
0x5AeFd5C04ed6DBd856A5aeB691eFcc80c0aB7472 107